<a href="https://colab.research.google.com/github/zbyju/machine-learning/blob/main/Copy_of_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install open3d


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import numpy as np
import open3d as o3d

import os
import time

import pytest
import numpy as np
import pandas as pd
import geopandas as gpd

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [9]:
print("Hello world")
print(np.array([1, 2, 3]))

Hello world
[1 2 3]


In [34]:
def loadXYZ(path):
  ply_point_cloud = o3d.data.PLYPointCloud()
  pcd = o3d.io.read_point_cloud(path, format='xyz')
  o3d.visualization.draw_geometries([pcd],
                                    zoom=0.3412,
                                    front=[0.4257, -0.2125, -0.8795],
                                    lookat=[2.6172, 2.0475, 1.532],
                                    up=[-0.0694, -0.9768, 0.2024])
  return np.asarray(pcd.points)

In [51]:
def getRawFeaturesFromData(points, treeType):
  df = pd.DataFrame(points)
  df.columns = ['x','y','z']

  pointCount = len(df.index)

  min_z = df["z"].min()
  max_z = df["z"].max()

  height = max_z - min_z
  width = 1

  # Cut-offs are: [min, third), [third, top - third), [top - third, top ]
  topCount = 0
  midCount = 0
  botCount = 0
  restCount = 0
  for index, row in df.iterrows():
    if(row['z'] >= min_z and row['z'] < third):
      botCount += 1
    elif(row['z'] >= third and row['z'] < max_z - third):
      midCount += 1
    elif(row['z'] >= max_z - third and row['z'] <= max_z):
      topCount += 1
    else:
      restCount += 1

  if restCount != 0:
    print("WARNING: Rest count is not zero! It is:", restCount)

  return (pointCount, height, width, topCount, midCount, botCount, treeType)

In [76]:
trainDir = "/content/data/train/"
treeTypes = ["pine", "spruce", "birch"]

rawData = pd.DataFrame({ "pointCount": [], "height": [], "width": [], "topCount": [], "midCount": [], "botCount": [], "treeType": []})

# Pines
for treeType in treeTypes:
  for filename in os.listdir(trainDir + treeType):
    f = os.path.join(pineDir, filename)
    if not os.path.isfile(f):
      print("WARNING: Not a file - ", f)
      continue
    
    points = loadXYZ(f)
    data = getRawFeaturesFromData(points, treeType)
    print(data)
    rawData.loc[len(rawData.index)] = data

print(rawData)
  

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
(126143, 10.934, 1, 60639, 53550, 11954, 'pine')
   pointCount  height  width  topCount  midCount  botCount treeType
0    126143.0  10.934    1.0   60639.0   53550.0   11954.0     pine


In [77]:
# Normalize data

min_height = rawData["height"].min()
max_height = rawData["height"].max()
if min_height == max_height:
  min_height = 0
min_width = rawData["width"].min()
max_width = rawData["width"].max()
if min_width == max_width:
  min_width = 0

d = pd.DataFrame()
d["tree"] = rawData["treeType"]
d["count"] = rawData["pointCount"]
d["height"] = rawData["height"].apply(lambda x: (x - min_height) / (max_height - min_height))
d["width"] = rawData["width"].apply(lambda x: (x - min_width) / (max_width - min_width))
d["top"] = rawData.apply(lambda x: x["topCount"] / x["pointCount"], axis=1)
d["mid"] = rawData.apply(lambda x: x["midCount"] / x["pointCount"], axis=1)
d["bot"] = rawData.apply(lambda x: x["botCount"] / x["pointCount"], axis=1)

print(d.head())


   tree     count  height  width       top       mid       bot
0  pine  126143.0     1.0    1.0  0.480716  0.424518  0.094765


# Method 1 - 